### **Table of Contents**
    
* [Reading in the data](#reading-in-the-data)
* [cleaning](#cleaning)
  * [Feature engineering](#feature-engineering)
* [Function list](#function-list)
* [Keep this file in your project](#keep-this-file-in-your-project)


In [1]:
import pandas as pd
from pandas.testing import assert_frame_equal
import numpy as np

# formatting notebook to show rounded values.
pd.options.display.float_format = '{:,.2f}'.format

# Reading in the data 

- Reading in the data 
- finding the data types for each column. 
- optimized each column to not have objects as they are more memory hungry.  

In [2]:
def read_salary_csv(filepath: str) -> pd.DataFrame:
    """
    Read a salary CSV file with optimized dtypes for memory efficiency.
    
    Parameters:
        filepath (str): Path to the CSV file.
        
    Returns:
        pd.DataFrame: Loaded DataFrame with optimized dtypes.
    """
    optimized_dtypes = {
        "CalYear": "int16",
        "Employee_Name": "string",
        "Department": "category",
        "jobTitle": "category",
        "Annual_Rate": "float32",
        "Regular_Rate": "float32",
        "Overtime_Rate": "float32",
        "Incentive_Allowance": "float32",
        "Other": "float32",
        "YTD_Total": "float32",
        "ObjectId": "int32"
    }
    
    sal = pd.read_csv(filepath, dtype=optimized_dtypes)
    return sal

# Example usage:
sal = read_salary_csv("data/salary2.csv")
sal.head()

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,Other,YTD_Total,ObjectId
0,2021,<NA>,Belle of Louisville,Hospitality Worker,"3,120.00",687.00,0.00,0.00,NaN,687.00,1
1,2021,<NA>,Parks & Recreation,Park Aide,"31,200.00",600.00,0.00,0.00,NaN,600.00,2
2,2021,"Martin, David",Library,Library Page L/U,"28,433.60","28,412.56",0.00,"1,450.00",NaN,"30,862.81",3
3,2021,"Bratcher, Elaine",Louisville Metro Police,Clerk Typist II-Police,"35,256.00","35,256.01",0.00,"1,563.12",NaN,"36,819.13",4
4,2021,"Jackson, Nila",Louisville Metro Police,Traffic Guard II,"21,418.80","16,529.14",952.39,"1,000.00",NaN,"18,481.53",5


# cleaning 

In [3]:
def clean_salaries(sal: pd.DataFrame) -> pd.DataFrame:
    """
    Fill missing values for specific columns and drop unnecessary columns.

    - Fills missing 'Employee_Name' with 'Unknown'
    - Fills missing 'Other' with 0
    - Drops the 'Other' column

    Returns the cleaned DataFrame.
    """
    # Fill missing values
    sal = sal.fillna({
        "Employee_Name": "Unknown",
        "Other": 0
    })

    # Drop 'Other' column
    if "Other" in sal.columns:
        sal = sal.drop("Other", axis=1)

    return sal

# Example usage
sal = clean_salaries(sal)
sal.head()

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,YTD_Total,ObjectId
0,2021,Unknown,Belle of Louisville,Hospitality Worker,"3,120.00",687.00,0.00,0.00,687.00,1
1,2021,Unknown,Parks & Recreation,Park Aide,"31,200.00",600.00,0.00,0.00,600.00,2
2,2021,"Martin, David",Library,Library Page L/U,"28,433.60","28,412.56",0.00,"1,450.00","30,862.81",3
3,2021,"Bratcher, Elaine",Louisville Metro Police,Clerk Typist II-Police,"35,256.00","35,256.01",0.00,"1,563.12","36,819.13",4
4,2021,"Jackson, Nila",Louisville Metro Police,Traffic Guard II,"21,418.80","16,529.14",952.39,"1,000.00","18,481.53",5


## Feature engineering
Calculating pay and hours for people. 

- Each function has a test to verify that it works as intended
- It calls a tiny data set and does the calculations

In [4]:
def hr_rate(sal):
  # Calculate Hourly Rate and apply minimum wage rule
  sal["Hr_Rate"] = sal["Annual_Rate"] / 2080 

  # If hourly rate is less than $7.25, replace it with $7.25 (minimum wage floor)
  sal["Hr_Rate"] = sal["Hr_Rate"].mask(sal["Hr_Rate"] < 7.25, 7.25)

  # Calculate Overtime Rate
  sal["Ot_Rate"] = sal["Hr_Rate"] * 1.5
  return sal

def hr_rate_test():
    # Test data
    df = pd.DataFrame({
        'Employee_Name': ['Steve', 'Bill'], 
        'Annual_Rate': [40000, 10000]
    })

    expected = pd.DataFrame({
        'Employee_Name': ['Steve', 'Bill'], 
        'Annual_Rate': [40000, 10000],
        'Hr_Rate': [19.23, 7.25],
        'Ot_Rate': [28.85, 10.88]
    })

    # Run function
    result = hr_rate(df.copy())

    # Round to 2 decimals for comparison
    result[["Hr_Rate", "Ot_Rate"]] = result[["Hr_Rate", "Ot_Rate"]].round(2)

    # Test
    assert_frame_equal(result, expected)
    print("✅ Test passed!")

hr_rate_test()

✅ Test passed!


In [5]:
def part_time_filter(sal):
  # Filter out part-time employees
  sal['Reg_Hours'] = sal["Annual_Rate"] / sal["Hr_Rate"]

  # Keep only full-time employees (≥ 2080 hours)
  sal = sal[sal['Reg_Hours'] >= 2080]
  return sal 

def part_time_test():
    # Test data
    df = pd.DataFrame({'Employee_Name': ['Steve', 'Bill'], 
                   'Annual_Rate': [40000, 10000],
                   'Hr_Rate': [19.23, 7.25],
                   'Ot_Rate': [28.84, 10.88]})

    expected = pd.DataFrame({'Employee_Name': ['Steve'], 
                   'Annual_Rate': [40000],
                   'Hr_Rate': [19.23],
                   'Ot_Rate': [28.84],
                   'Reg_Hours': [2080]})

    # Run function
    result = part_time_filter(df.copy())

    # Round to 2 decimals for comparison
    result[["Hr_Rate", "Ot_Rate", 'Reg_Hours']] = result[["Hr_Rate", "Ot_Rate", 'Reg_Hours']].round(2)

    # Test
    assert_frame_equal(result, expected)
    print("✅ Test passed!")

hr_rate_test()

✅ Test passed!


In [6]:
def hour_calc(sal):
  # 4️ Calculate regular hours worked (based on Regular_Rate)
  sal['Rg_Hours'] = sal["Regular_Rate"] / sal["Hr_Rate"]

  # 5️ Calculate overtime hours worked
  sal['Ot_Hours'] = sal["Overtime_Rate"] / sal["Ot_Rate"]

  # 6️ Total hours worked
  sal['Total_Hours'] = sal['Reg_Hours'] + sal['Ot_Hours']

  # 7️ Estimate number of weeks worked
  sal["Weeks_Worked"] = sal["Regular_Rate"] / (sal["Hr_Rate"] * 40) 
  return sal


def hour_calc_test():
    # Test data
    df = pd.DataFrame({
       'Employee_Name': ['Steve'], 
        'Annual_Rate': [40000],
        'Regular_Rate': [40000],
        'Overtime_Rate': [288.40],
        'Hr_Rate': [19.23],
        'Ot_Rate': [28.84],
        'Reg_Hours': [2080]})

    expected = pd.DataFrame({
        'Employee_Name': ['Steve'],
        'Annual_Rate': [40000],
        'Regular_Rate': [40000],
        'Overtime_Rate': [288.40],
        'Hr_Rate': [19.23],
        'Ot_Rate': [28.84],
        'Reg_Hours': [2080],
        'Rg_Hours': [2080.08],
        'Ot_Hours': [10.0],
        'Total_Hours': [2090.00],
        'Weeks_Worked': [52.00]})

    # Run function
    result = hour_calc(df.copy())

    # Round all numeric columns to 2 decimals for comparison
    numeric_cols = result.select_dtypes(include='number').columns
    result[numeric_cols] = result[numeric_cols].round(2)

    # Test
    assert_frame_equal(result, expected)
    print("✅ Test passed!")


# Run the test
hour_calc_test()

✅ Test passed!


In [7]:
def week_calc(sal):
  # 8️ Regular hours per week
  sal["Reg_Hrs_per_Week"] = sal["Regular_Rate"] / (sal["Hr_Rate"] * sal["Weeks_Worked"])

  # 9️ Overtime hours per week
  sal["Ot_Hrs_per_Week"] = sal["Overtime_Rate"] / (sal["Ot_Rate"] * sal["Weeks_Worked"])

  # 10 Average hours per week
  sal["Avg_Hrs_per_Week"] = sal["Reg_Hrs_per_Week"] + sal["Ot_Hrs_per_Week"]
  return sal

def week_calc_test():
    # Test data
    df = pd.DataFrame({
       'Employee_Name': ['Steve'], 
        'Annual_Rate': [40000],
        'Regular_Rate': [40000],
        'Overtime_Rate': [288.40],
        'Hr_Rate': [19.23],
        'Ot_Rate': [28.84],
        'Reg_Hours': [2080]})

    expected = pd.DataFrame({
        'Employee_Name': ['Steve'],
        'Annual_Rate': [40000],
        'Regular_Rate': [40000],
        'Overtime_Rate': [288.40],
        'Hr_Rate': [19.23],
        'Ot_Rate': [28.84],
        'Reg_Hours': [2080],
        'Rg_Hours': [2080.08],
        'Ot_Hours': [10.0],
        'Total_Hours': [2090.00],
        'Weeks_Worked': [52.00],
        'Reg_Hrs_per_Week': [40.00],
        'Ot_Hrs_per_Week': [0.19],
        'Avg_Hrs_per_Week': [40.19],})

    # Run function
    result = week_calc(df.copy())

    # Round all numeric columns to 2 decimals for comparison
    numeric_cols = result.select_dtypes(include='number').columns
    result[numeric_cols] = result[numeric_cols].round(2)

    # Test
    assert_frame_equal(result, expected)
    print("✅ Test passed!")


# Run the test
hour_calc_test()

✅ Test passed!


---

# Function list 

``` python 
hr_rate(sal)
part_time_filter(sal)
hour_calc(sal)
week_calc(sal)
```

In [8]:
sal = hr_rate(sal)
sal = part_time_filter(sal)
sal = hour_calc(sal)
sal = week_calc(sal)

In [9]:
sal.head()

,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,YTD_Total,ObjectId,Hr_Rate,Ot_Rate,Reg_Hours,Rg_Hours,Ot_Hours,Total_Hours,Weeks_Worked,Reg_Hrs_per_Week,Ot_Hrs_per_Week,Avg_Hrs_per_Week
1,2021,Unknown,Parks & Recreation,Park Aide,"31,200.00",600.00,0.00,0.00,600.00,2,15.00,22.50,"2,080.00",40.00,0.00,"2,080.00",1.00,40.00,0.00,40.00
2,2021,"Martin, David",Library,Library Page L/U,"28,433.60","28,412.56",0.00,"1,450.00","30,862.81",3,13.67,20.51,"2,080.00","2,078.46",0.00,"2,080.00",51.96,40.00,0.00,40.00
3,2021,"Bratcher, Elaine",Louisville Metro Police,Clerk Typist II-Police,"35,256.00","35,256.01",0.00,"1,563.12","36,819.13",4,16.95,25.43,"2,080.00","2,080.00",0.00,"2,080.00",52.00,40.00,0.00,40.00
4,2021,"Jackson, Nila",Louisville Metro Police,Traffic Guard II,"21,418.80","16,529.14",952.39,"1,000.00","18,481.53",5,10.30,15.45,"2,080.00","1,605.16",61.66,"2,141.66",40.13,40.00,1.54,41.54
5,2021,"Ammon, Darrell",Louisville Metro Police,Criminal Justice Specialist,"50,107.20","49,362.47",0.00,0.00,"49,362.47",6,24.09,36.14,"2,080.00","2,049.09",0.00,"2,080.00",51.23,40.00,0.00,40.00


# Keep this file in your project

keep at the bottom of your project

We will go over it later, its a script that will generate a markdown table of contents based on your headings created here in markdown.

In [11]:
import json
import os


def generate_toc_from_notebook(notebook_path):
    """
    Parses a local .ipynb file and generates Markdown for a Table of Contents.
    """
    if not os.path.isfile(notebook_path):
        print(f"❌ Error: File not found at '{notebook_path}'")
        return

    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = json.load(f)

    toc_markdown = "### **Table of Contents**\n"
    for cell in notebook.get('cells', []):
        if cell.get('cell_type') == 'markdown':
            for line in cell.get('source', []):
                if line.strip().startswith('#'):
                    level = line.count('#')
                    title = line.strip('#').strip()
                    link = title.lower().replace(' ', '-').strip('-.()')
                    indent = '  ' * (level - 1)
                    toc_markdown += f"{indent}* [{title}](#{link})\n"

    print("\n--- ✅ Copy the Markdown below and paste"
          "it into a new markdown cell ---\n")
    print(toc_markdown)


if __name__ == "__main__":
    # Example usage
    notebook_path = 'dannyRefactor.ipynb'  # Replace with your notebook path
    generate_toc_from_notebook(notebook_path)


--- ✅ Copy the Markdown below and pasteit into a new markdown cell ---

### **Table of Contents**
    * [**Table of Contents**](#**table-of-contents**)
* [Reading in the data](#reading-in-the-data)
* [cleaning](#cleaning)
  * [Feature engineering](#feature-engineering)
* [Function list](#function-list)
* [Keep this file in your project](#keep-this-file-in-your-project)

